In [0]:
# Databricks: 02_transform_silver.py
# Purpose: Create cleaned "silver_*" views from Bronze tables.
# - Works with Unity Catalog (set CATALOG) or non-UC (leave CATALOG empty).
# - Optional de-duplication for payments/chargebacks via ROW_NUMBER.

# -----------------------------
# Widgets / Parameters
# -----------------------------
dbutils.widgets.text("CATALOG", "")               # e.g., "main" if UC is enabled; leave blank for non-UC
dbutils.widgets.text("SCHEMA", "demo_finance")    # target schema/database
dbutils.widgets.dropdown("DEDUP_STRATEGY", "none", ["none", "keyed"])
# DEDUP_STRATEGY:
#   none  -> no dedupe
#   keyed -> keep latest per natural key (payment_id for payments; payment_id+opened_ts for chargebacks)

# -----------------------------
# Read widgets
# -----------------------------
CATALOG        = dbutils.widgets.get("CATALOG").strip()
SCHEMA         = dbutils.widgets.get("SCHEMA").strip()
DEDUP_STRATEGY = dbutils.widgets.get("DEDUP_STRATEGY").strip().lower()

if not SCHEMA:
    raise ValueError("SCHEMA must not be empty")

# -----------------------------
# Helpers
# -----------------------------
def fqtn(table: str) -> str:
    """Fully-qualified table/view name."""
    if CATALOG:
        return f"{CATALOG}.{SCHEMA}.{table}"
    else:
        return f"{SCHEMA}.{table}"

def use_schema():
    if CATALOG:
        spark.sql(f"USE CATALOG {CATALOG}")
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA}")
    spark.sql(f"USE SCHEMA {SCHEMA}")

# -----------------------------
# Use catalog/schema and verify Bronze exists
# -----------------------------
use_schema()

bronze_tables = ["bronze_subscriptions", "bronze_payments", "bronze_chargebacks"]
missing = []
for t in bronze_tables:
    try:
        _ = spark.table(fqtn(t)).limit(1).count()
    except Exception:
        missing.append(fqtn(t))

if missing:
    raise RuntimeError(
        "Missing required Bronze tables:\n  - " + "\n  - ".join(missing) +
        "\nRun 01_ingest_bronze.py first (same CATALOG/SCHEMA)."
    )

# -----------------------------
# Build Silver: Subscriptions
# -----------------------------
spark.sql(f"""
CREATE OR REPLACE VIEW {fqtn('silver_subscriptions')} AS
SELECT
  CAST(customer_id AS STRING)                      AS customer_id,
  CAST(plan        AS STRING)                      AS plan,
  TO_DATE(start_date)                              AS start_date,
  CASE WHEN end_date IS NULL OR end_date = '' THEN NULL ELSE TO_DATE(end_date) END AS end_date,
  CAST(amount      AS DOUBLE)                      AS amount,
  CAST(status      AS STRING)                      AS status
FROM {fqtn('bronze_subscriptions')}
""")

# -----------------------------
# Build Silver: Payments (optional dedupe)
# Natural key: payment_id (keep the latest by ts)
# -----------------------------
if DEDUP_STRATEGY == "keyed":
    payments_sql = f"""
    CREATE OR REPLACE VIEW {fqtn('silver_payments')} AS
    WITH typed AS (
      SELECT
        CAST(payment_id AS STRING)   AS payment_id,
        CAST(customer_id AS STRING)  AS customer_id,
        CAST(amount     AS DOUBLE)   AS amount,
        CAST(status     AS STRING)   AS status,
        CAST(method     AS STRING)   AS method,
        to_timestamp(ts)             AS ts,
        date_format(to_timestamp(ts), 'yyyy-MM') AS month
      FROM {fqtn('bronze_payments')}
    ),
    ranked AS (
      SELECT *, ROW_NUMBER() OVER (PARTITION BY payment_id ORDER BY ts DESC, amount DESC) AS rn
      FROM typed
    )
    SELECT payment_id, customer_id, amount, status, method, ts, month
    FROM ranked
    WHERE rn = 1
    """
else:
    payments_sql = f"""
    CREATE OR REPLACE VIEW {fqtn('silver_payments')} AS
    SELECT
      CAST(payment_id AS STRING)   AS payment_id,
      CAST(customer_id AS STRING)  AS customer_id,
      CAST(amount     AS DOUBLE)   AS amount,
      CAST(status     AS STRING)   AS status,
      CAST(method     AS STRING)   AS method,
      to_timestamp(ts)             AS ts,
      date_format(to_timestamp(ts), 'yyyy-MM') AS month
    FROM {fqtn('bronze_payments')}
    """

spark.sql(payments_sql)

# -----------------------------
# Build Silver: Chargebacks (optional dedupe)
# Natural key: (payment_id, opened_ts) (keep the latest by resolved_ts)
# -----------------------------
if DEDUP_STRATEGY == "keyed":
    chargebacks_sql = f"""
    CREATE OR REPLACE VIEW {fqtn('silver_chargebacks')} AS
    WITH typed AS (
      SELECT
        CAST(payment_id AS STRING)   AS payment_id,
        CAST(reason     AS STRING)   AS reason,
        CAST(outcome    AS STRING)   AS outcome,
        to_timestamp(opened_ts)      AS opened_ts,
        to_timestamp(resolved_ts)    AS resolved_ts,
        date_format(to_timestamp(opened_ts), 'yyyy-MM') AS month_opened
      FROM {fqtn('bronze_chargebacks')}
    ),
    ranked AS (
      SELECT *,
             ROW_NUMBER() OVER (
               PARTITION BY payment_id, opened_ts
               ORDER BY resolved_ts DESC NULLS LAST
             ) AS rn
      FROM typed
    )
    SELECT payment_id, reason, outcome, opened_ts, resolved_ts, month_opened
    FROM ranked
    WHERE rn = 1
    """
else:
    chargebacks_sql = f"""
    CREATE OR REPLACE VIEW {fqtn('silver_chargebacks')} AS
    SELECT
      CAST(payment_id AS STRING)   AS payment_id,
      CAST(reason     AS STRING)   AS reason,
      CAST(outcome    AS STRING)   AS outcome,
      to_timestamp(opened_ts)      AS opened_ts,
      to_timestamp(resolved_ts)    AS resolved_ts,
      date_format(to_timestamp(opened_ts), 'yyyy-MM') AS month_opened
    FROM {fqtn('bronze_chargebacks')}
    """

spark.sql(chargebacks_sql)

# -----------------------------
# Basic QA / Row counts
# -----------------------------
counts = {}
for v in ["silver_subscriptions", "silver_payments", "silver_chargebacks"]:
    cnt = spark.table(fqtn(v)).count()
    counts[v] = cnt

print("Row counts (Silver):")
for k, v in counts.items():
    print(f"  {fqtn(k)} -> {v}")

# Null checks on key fields
null_checks = {
    "silver_subscriptions.customer_id": f"SELECT COUNT(*) c FROM {fqtn('silver_subscriptions')} WHERE customer_id IS NULL",
    "silver_payments.payment_id":       f"SELECT COUNT(*) c FROM {fqtn('silver_payments')} WHERE payment_id IS NULL",
    "silver_chargebacks.payment_id":    f"SELECT COUNT(*) c FROM {fqtn('silver_chargebacks')} WHERE payment_id IS NULL"
}
for label, sql in null_checks.items():
    c = spark.sql(sql).collect()[0]["c"]
    print(f"Nulls in {label}: {c}")

print("✅ Silver transform complete.")